# Ball Segmentation - Inference Demo

This notebook demonstrates how to use trained models for inference on new images.

In [ ]:
from pathlib import Path
from time import perf_counter

import torch
from ultralytics import YOLO
from IPython.display import Image as IPImage, display

In [ ]:
MODEL_PATH = Path('models/trained/ball_person_best.pt')
TEST_IMAGES_DIR = Path('df')
DEVICE = 'cuda:0' if torch.cuda.is_available() else 'cpu'

print(f"Device: {DEVICE}")

## Load Model

In [ ]:
model = YOLO(str(MODEL_PATH))

## Run Inference on Single Image

In [ ]:
img_path = TEST_IMAGES_DIR / "test.jpg"

results = model.predict(
    source=str(img_path),
    save=True,
    conf=0.25,
    iou=0.5,
    imgsz=640,
    device=DEVICE,
    show_labels=True,
    show_conf=True,
    project='runs/inference',
    name='demo',
    exist_ok=True
)

pred_path = Path('runs/inference/demo') / img_path.name
if pred_path.exists():
    display(IPImage(filename=str(pred_path), width=800))